In [ ]:
# Ozer Guney
# Kha
# Lucky
# Roman

In [1]:
import os
import pandas as pd
import numpy as np
import Levenshtein

In [3]:
# Ce dossier a été nettoyé manuellement pour enlever des éléments dans certains fichiers. 
# Par exemple: 
# fichier 10: (details)
# fichier 11: les valeurs comme 0.5-1.0 sont remplacées par 0.75, la moyenne
# fichier 27: (OPEC)
# fichier 31: [Note]
# fichier 33: (more info)
# etc. ...
dossier_brutes = 'Données brutes'

In [4]:
# Nettoyage des données brutes
# Les lignes contenant des valeurs vides, manquantes, des % et des < ont été supprimées
# Certains noms de pays ont été aussi changés pour correspondre au nom des pays dans le fichier countries of the world
for fichier in os.listdir(dossier_brutes):
    if fichier.endswith('.csv'):
        chemin_complet = os.path.join(dossier_brutes, fichier)
        
        # Lire le fichier CSV
        df = pd.read_csv(chemin_complet)
        
        # Vérifier que la deuxième colonne existe
        if df.shape[1] >= 2:
                                

            # Traiter chaque ligne
            supprimer_lignes = []
            for index in range(len(df)):

                cell_value = df.iloc[index, 1]
                # Vérifier si la deuxième colonne est vide 
                if pd.isna(cell_value):
                    supprimer_lignes.append(index)
                    continue  
                
                str_cell_value = str(cell_value)
                if str_cell_value== '—' or str_cell_value=='–' or str_cell_value=='-':
                    supprimer_lignes.append(index)
                    continue

                if str_cell_value == 'n.a.':
                    supprimer_lignes.append(index)
                    continue        
                
                if isinstance(cell_value, str):
                    df.iloc[index, 1] = cell_value.replace(',', '')
                    
                cell_value = df.iloc[index, 1]
                str_cell_value = str(cell_value)
                if str_cell_value.strip().startswith('<'):
                    df.iloc[index, 1] = str_cell_value.replace('< ', '')
                    
                cell_value = df.iloc[index, 1]
                str_cell_value = str(cell_value)
                if str_cell_value.strip().endswith('%'):
                    df.iloc[index, 1] = str_cell_value.rstrip('%')

                    
            df = df.drop(supprimer_lignes)
            # Enregistrer les modifications
            
            nouveau_chemin = os.path.join('Données nettoyées', fichier)
            df.to_csv(nouveau_chemin, index=False)
            
            for index in range(len(df)):
                cell_value = df.iloc[index, 0]
                if str(cell_value) == 'DR Congo' or str(cell_value) == 'Congo DR' or str(cell_value) == 'Congo, Democratic Republic of':    
                    df.iloc[index, 0] = 'Democratic Republic of the Congo'
                if str(cell_value) == 'Congo, Republic of':    
                    df.iloc[index, 0] = 'Republic of the Congo'    
                if str(cell_value) == 'Saint Helena, Ascension and Tristan da Cunha' or str(cell_value) == 'Saint Helena, Ascension, and Tristan da Cunha':  
                    df.iloc[index, 0] = 'Saint Helena'
                
            df.to_csv(nouveau_chemin, index=False)

In [ ]:
# Calculer les statistiques 

dossier_nettoyé = 'Données nettoyées'
fichiers = os.listdir(dossier_nettoyé)
fichiers.sort()

stats_list = []	
for filename in fichiers:
    if filename.endswith('.csv'):
        chemin_complet = os.path.join(dossier_nettoyé, filename)
        # Lire le fichier CSV
        df = pd.read_csv(chemin_complet)

        # Calculer les statistiques pour la deuxième colonne
        column = df.iloc[1:, 1]  # La deuxième colonne
        stats_dict = {
            'filename': filename,
            'mean': column.mean(),
            'median': column.median(),
            'mode': column.mode()[0],
            'max': column.max(),
            'min': column.min(),
            'variance': column.var()
        }
        print(stats_dict)
        # Ajouter le dictionnaire à la liste
        stats_list.append(stats_dict)


In [6]:
# Binarisé les données nettoyées

i = 0
fichiers = os.listdir(dossier_nettoyé)
fichiers.sort()

for fichier in fichiers:
    if fichier.endswith('.csv'):
        chemin_complet = os.path.join(dossier_nettoyé, fichier)
        median = stats_list[i]['median']
        # Lire le fichier CSV
        df = pd.read_csv(chemin_complet) # chaque fichier

        if df.shape[1] >= 2:
            # Traiter chaque ligne
            for index in range(len(df)):
                cell_value = df.iloc[index, 1]
                if cell_value > median:
                    df.iloc[index, 1] = 1
                else:
                    df.iloc[index, 1] = 0
                
            # Enregistrer les modifications dans un nouveau dossier
            df.to_csv(os.path.join('Données binarisées', fichier), index=False)
    i = i + 1

In [7]:

# Création du Fichier finale

n_cols = 41
n_rows = 238
df = pd.DataFrame(np.nan, index=range(n_rows), columns=range(n_cols))

# Enregistrer le DataFrame dans un fichier CSV
df.to_csv('fichier_finale.csv', index=False)

In [11]:
# Placer tous les pays qui existent dans le fichier finale.csv à partir du fichier countries of the world.csv

fichier_finale = 'fichier_finale.csv'
countries = 'countries of the world.csv'
df1 = pd.read_csv(countries)
df2 = pd.read_csv(fichier_finale)

colonne = df1['Country']

# Assurez-vous que le nombre de lignes dans les deux DataFrames correspond
df2.rename(columns={'0': 'Country'}, inplace=True)
df2['Country'] = colonne

# Enregistrer le second DataFrame dans un nouveau fichier CSV
df2.to_csv(fichier_finale, index=False)

In [13]:
#Renommer les colonnes du fichier finale.csv par les nom des colonnes des 40 fichiers

dossier_binarisé = 'Données binarisées'
fichiers_binarisé = os.listdir(dossier_binarisé)
fichiers_binarisé.sort()

i = 1
for fichier in fichiers:
    df2 = pd.read_csv(fichier_finale)
    if fichier.endswith('.csv'):
        chemin_complet = os.path.join(dossier_binarisé, fichier)
        df1 = pd.read_csv(chemin_complet)
        colonne = df1.columns[1]
        df2.rename(columns={str(i): colonne}, inplace=True)

        # Lire le fichier CSV

                    
    df2.to_csv(fichier_finale, index=False)    
    i = i + 1    
        
             
        

In [14]:
# Remplir les valeurs du fichier finale.csv par les valeurs des 40 fichiers du dossier Données binarisées
# Utiliser la distance de Levenshtein pour trouver les pays similaires et remplacer les valeurs
# Peux prendre du temps pour s'exécuter (environ 20 secondes)

df2 = pd.read_csv(fichier_finale)
    # Parcourir tous les fichiers dans le dossier
i = 1
for fichier in fichiers_binarisé:
    if fichier.endswith('.csv'):
        chemin_complet = os.path.join(dossier_binarisé, fichier)
        
        # Lire le fichier CSV
        df1 = pd.read_csv(chemin_complet)

        for index in range(len(df1)):
            # Utiliser .iloc pour accéder à la valeur par position
            cell_pays = df1.iloc[index, 0]
            cell_valeur = df1.iloc[index, 1]
            temp_similarite = 0
            temp_index = 0
            
            for index2 in range(len(df2)):
                cell_pays2 = df2.iloc[index2, 0]
                
                # Comparer les valeurs des cellules
                if Levenshtein.ratio(cell_pays, cell_pays2) > 0.95:
                    df2.iloc[index2, i] = cell_valeur
                    break
                if Levenshtein.ratio(cell_pays, cell_pays2) > 0.74 and Levenshtein.ratio(cell_pays, cell_pays2) > temp_similarite:
                    # Remplacer la valeur de la cellule

                    temp_index = index2
                    temp_similarite = Levenshtein.ratio(cell_pays, cell_pays2)
                    temp_cell_pays = cell_pays
                      
            if temp_similarite> 0.74:
                df2.iloc[temp_index, i] = cell_valeur
    
    i = i + 1    
            # Enregistrer les modifications
    df2.to_csv(fichier_finale, index=False)